# Create Search Result & Reservation Fact tables from sources
## Take Bronze data and load it into Silver

In [0]:
srSQL = '''
CREATE OR REPLACE TABLE neighbor.silver.searchevents AS
SELECT 
  ca.CalendarKey,
  t.TimeKey,
  sa.merged_amplitude_id as MergedAmplitudeID,
  sa.search_id as SearchID,
  sa.listingID,
  dma.DMAID,
  st.SearchTermID,
  ch.channelID,
  src.SourceID,
  sa.prior_search_id as priorsearchid,
  avg(SearchResults) AS SearchCounts,
  count(lde.search_id) as PropertiesClicked,
  round(avg (lde.search_position),0) as AvgSearchPositionClicked,
  to_utc_timestamp(CURRENT_TIMESTAMP, 'UTC') AS UPDATEDATEUTC
FROM neighbor.bronze.all_searcheventsagg sa
JOIN neighbor.silver.calendar ca 
  ON sa.event_date = ca.CalendarDate
JOIN neighbor.silver.time t 
  ON sa.event_time >= t.StartTime
     AND sa.event_time < t.EndTime
JOIN neighbor.silver.dma dma 
  ON sa.search_dma = dma.DMAName
JOIN neighbor.silver.searchterm st
  ON sa.search_term = st.SearchTerm 
     AND sa.search_term_category = st.SearchTermCategory 
     AND sa.search_type = st.SearchType  
JOIN neighbor.silver.channel ch
  ON sa.first_attribution_channel = ch.channelName
JOIN neighbor.silver.source src 
  ON sa.first_attribution_source = src.SourceName
JOIN neighbor.bronze.listing_detail_events lde
  ON lde.search_id = sa.search_id
GROUP BY 
  ca.CalendarKey,
  t.TimeKey,
  sa.merged_amplitude_id,
  sa.search_id,
  sa.listingID,
  dma.DMAID,
  st.SearchTermID,
  ch.channelID,
  src.SourceID,
  sa.prior_search_id
'''
spark.sql(srSQL)

In [0]:
%sql
SELECT * FROM neighbor.silver.searchevents

In [0]:
resSQL ='''
CREATE OR REPLACE TABLE neighbor.silver.reservations AS
SELECT 
LDV.*,
resv.AvgSearchPosition,
resv.Reserved+resv.NotReserved as TotalSearchCount,
resv.Reserved,
resv.NotReserved
from
(select distinct
 concat(d.DMAID,'-',lde.listing_id) as ListingDMA,
 d.DMAName,
 lde.listing_id as ListingID,
 first_attribution_source as FirstAttributionSource,
 first_attribution_channel FirstAttributionChannel,
 count(1) as AppearedInSearches
 from neighbor.bronze.listing_detail_events lde
 LEFT JOIN  neighbor.silver.dma d on lde.click_dma=d.DMAName
 where lde.is_bot='false' and is_host='false'
group by 
concat(d.DMAID,'-',lde.listing_id) ,
d.dmaName,
 lde.listing_id,
 first_attribution_source,
 first_attribution_channel
)LDV
LEFT JOIN 
(
select distinct
concat(d.DMAID,'-',lde.listing_id) as DMAListingID,
first_attribution_source, 
first_attribution_channel,
round(avg(lde.search_position),0) as AvgSearchPosition,
sum(case when r.created_at is not null then 1 else 0 end) as Reserved,
sum(case when r.created_at is null then 1 else 0 end) as NotReserved
 from neighbor.bronze.listing_detail_events lde
join neighbor.silver.dma d on lde.click_dma=d.DMAName
left join neighbor.bronze.reservations r on lde.listing_id=r.listing_id and lde.click_dma=r.dma
and cast(lde.event_date as date) = cast(r.created_at as date)
where lde.is_bot='false' and is_host='false'
group by 
concat(d.DMAID,'-',lde.listing_id),
first_attribution_source, 
first_attribution_channel
)resv
on LDV.ListingDMA=resv.DMAListingID
and LDV.FirstAttributionSource=resv.first_attribution_source and LDV.FirstAttributionChannel=resv.first_attribution_channel

'''
spark.sql(resSQL)